## CALCULATE AVERAGE THRESHOLD IN STELLAR LATEST USED

In [2]:
import re
import pandas as pd
import sys

log_path = 'src/ledger_logs.txt'
pattern = re.compile(r"needed=(\d+)")
needed = []

with open(log_path) as f:
    for line in f:
        m = pattern.search(line)
        if m:
            needed.append(int(m.group(1)))

df_needed = pd.DataFrame({'needed': needed})
avg_needed = df_needed['needed'].mean()

display(df_needed)

FileNotFoundError: [Errno 2] No such file or directory: 'src/ledger_logs.txt'

In [ ]:
print(f"Average 'needed' (minimum_quorum) across all checks: {avg_needed:.2f}")

In [6]:
import json
import pandas as pd

json_path = 'src/quorumset_05_06_2025.json'
with open(json_path) as f:
    raw = json.load(f)

def flatten_list(inner):
    result = []
    for v in inner.get('validators', []):
        result.append(v)
    for sub in inner.get('innerQuorumSets', []):
        result.extend(flatten_list(sub))
    return result

rows = []
for entry in raw['nodes']:
    pk = entry['publicKey']
    qs = entry['quorumSet']
    thr = qs['threshold']
    all_ids = set(qs.get('validators', []))
    for sub in qs.get('innerQuorumSets', []):
        all_ids.update(flatten_list(sub))
    peer_count = len(all_ids)
    threshold_percent = thr / peer_count if peer_count > 0 else None
    rows.append({
        'node': pk,
        'peer_count': peer_count,
        'threshold': thr,
        'threshold_percent': threshold_percent
    })

df_peers = pd.DataFrame(rows)
avg_peers = df_peers['peer_count'].mean()
avg_percent = df_peers['threshold_percent'].dropna().mean()

display(df_peers)


,node,peer_count,threshold,threshold_percent
0,GAAV2GCVFLNN522ORUYFV33E76VPC22E72S75AQ6MBR5V4...,21,5,0.238095
1,GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7Z...,23,5,0.217391
2,GCGB2S2KGYARPVIA37HYZXVRM2YZUEXA6S33ZU5BUDC6TH...,21,5,0.238095
3,GABMKJM6I25XI4K7U6XWMULOUQIQ27BCTMLS6BYYSOWKTB...,21,5,0.238095
4,GCM6QMP3DLRPTAZW2UZPCPX2LF3SXWXKPMP3GKFZBDSF3Q...,21,5,0.238095
...,...,...,...,...
100,GDRCZ4IPJR7V3HK4GR45CRTE72SDAOZUF2TDBQ5E5IGWC4...,32,7,0.218750
101,GATGABTRCJT76L63PXOBJ5MPJBRFEFUIPYO3FCQ3GGFL3U...,21,5,0.238095
102,GDH7HKT3XWIBJKC4PNQ6GHMWYTCY75WM4ENY2SSTMNTYLJ...,26,6,0.230769
103,GA5A7OIAFB4TCVJNPJRAQKBDAAIAC6EKCTNXVGF3F2KKFK...,22,5,0.227273


In [7]:
print(f"Average number of unique peers per node: {avg_peers:.2f}")
print(f"Average threshold as a fraction of peers: {avg_percent:.2%}")

Average number of unique peers per node: 41.77
Average threshold as a fraction of peers: 18.76%


In [ ]:
import json
import pandas as pd

json_path = 'src/07_2024_Real_SCP_network_topology.json'
with open(json_path) as f:
    raw = json.load(f)

def flatten_list(inner):
    # collect *validators* itself, but for slice-count we don't flatten deeper
    return inner.get('validators', [])

rows = []
for entry in raw['nodes']:
    pk = entry['publicKey']
    qs = entry['quorumSet']
    thr = qs['threshold']
    top_validators    = len(qs.get('validators', []))
    top_slices        = 1                           # the top-level slice itself counts as 1
    inner_slices      = len(qs.get('innerQuorumSets', []))
    total_slices      = top_slices + inner_slices
    # what % of slices does JSON-threshold represent?
    threshold_pct     = thr / total_slices if total_slices>0 else None

    rows.append({
        'node'             : pk,
        'total_slices'     : total_slices,
        'json_threshold'   : thr,
        'threshold_percent': threshold_pct
    })

df = pd.DataFrame(rows)

print(f"avg. slices/node = {df['total_slices'].mean():.1f}")
print(f"avg. JSON threshold = {df['json_threshold'].mean():.1f}")
print(f"avg. % of slices = {100*df['threshold_percent'].mean():.1f}%")


# CODE FOR OLD SNAPSHOT

In [5]:
import json
import pandas as pd

json_path = 'src/quorumset_05_06_2025.json'
with open(json_path) as f:
    raw = json.load(f)

def flatten_validators(qs):
    # Recursively collect all validators in this quorum set and its inner sets
    validators = set(qs.get('validators', []))
    for sub in qs.get('innerQuorumSets', []):
        validators |= flatten_validators(sub)
    return validators

rows = []
all_node_pks = set()

for entry in raw['nodes']:
    pk = entry['publicKey']
    all_node_pks.add(pk)

    # This node's direct validators + all nested ones
    all_peers = set(entry.get('validators', []))
    for sub in entry.get('innerQuorumSets', []):
        all_peers |= flatten_validators(sub)
    peer_count = len(all_peers)

    thr = entry['threshold']
    threshold_percent = thr / peer_count if peer_count > 0 else None

    rows.append({
        'node': pk,
        'peer_count': peer_count,
        'threshold': thr,
        'threshold_percent': threshold_percent
    })

df_peers = pd.DataFrame(rows)

avg_peers = df_peers['peer_count'].mean()
avg_percent = df_peers['threshold_percent'].dropna().mean()

print(f"Total unique nodes in network: {len(all_node_pks)}")
print(f"Avg. peers per node: {avg_peers:.2f}")
print(f"Avg. threshold percent: {avg_percent:.2f}")

#display(df_peers)


KeyError: 'threshold'